# CycleGAN Notebook for Style Transfer
## In this notebook, CycleGAN is used to generate images from different domains.
### Experimented Domains:
- Real building Architecture to Steampunk building design
- Season generation

Note: This notebook is based heavily on the CycleGAN Repositiory by https://github.com/xhujoy/CycleGAN-tensorflow


In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

import os, math
import time
import tensorflow as tf
import numpy as np
from glob import glob

import argparse
from progressbar import ProgressBar
tf.set_random_seed(19)

from module import *
from utils import *
from collections import namedtuple

In [2]:
print(tf.__version__)

1.3.0


### Defining Parameters

In [3]:
dataset_dir = './Steampunk2building/'   # Dataset dir
checkpoint_dir = './checkpoints/'  # checkpoint_dir
result_dir = './results/'          # result_dir
log_dir = './logs/'                # log_dir
sample_dir = './sample/'           # sample_dir

epoches = 200                     # Number of Epoches
batch_size = 1                     # Batch_size
epoch_step = 100                   # Number of epoch to decay lr
init_learn_rate = 0.0002           # Initial learning rate for adam
beta1 = 0.5
L1_lambda = 10.0
max_pool = 50                      # Max size of image pool

# input parameters
input_height = 256
input_width = 256
input_nc =3
output_height = 256
output_width = 256
output_nc = 3

ngf = 64
ndf= 64

save_freq = 10
print_freq = 10

# test

parser = argparse.ArgumentParser(description='')
parser.add_argument('--train_size', dest='train_size', type=int, default=1e8, help='# images used to train')
parser.add_argument('--load_size', dest='load_size', type=int, default=286, help='scale images to this size')
parser.add_argument('--fine_size', dest='fine_size', type=int, default=256, help='then crop to this size')
parser.add_argument('--beta1', dest='beta1', type=float, default=0.5, help='momentum term of adam')
parser.add_argument('--which_direction', dest='which_direction', default='AtoB', help='AtoB or BtoA')
parser.add_argument('--phase', dest='phase', default='train', help='train, test')
parser.add_argument('--save_freq', dest='save_freq', type=int, default=1000, help='save a model every save_freq iterations')
parser.add_argument('--print_freq', dest='print_freq', type=int, default=100, help='print the debug information every print_freq iterations')
parser.add_argument('--continue_train', dest='continue_train', type=bool, default=False, help='if continue training, load the latest model: 1: true, 0: false')
parser.add_argument('--test_dir', dest='test_dir', default='./test', help='test sample are saved here')


_StoreAction(option_strings=['--test_dir'], dest='test_dir', nargs=None, const=None, default='./test', type=None, choices=None, help='test sample are saved here', metavar=None)

In [4]:
def save(self, checkpoint_dir, step):
    model_name = "cyclegan.model"
    model_dir = "%s_%s" % (self.dataset_dir, self.image_size)
    checkpoint_dir = os.path.join(checkpoint_dir, model_dir)

    if not os.path.exists(checkpoint_dir):
        os.makedirs(checkpoint_dir)

    saver.save(sess,
                    os.path.join(checkpoint_dir, model_name),
                    global_step=step)

In [5]:
tfconfig = tf.ConfigProto(allow_soft_placement=True)
tfconfig.gpu_options.allow_growth = True

# with tf.Session(config=tfconfig) as sess:
sess = tf.Session(config=tfconfig)
#model = cyclegan(sess, args)

discriminator = discriminator
generator = generator_resnet # generator_unet
criterionGAN = mae_criterion # abs_criterion

OPTIONS = namedtuple('OPTIONS', 'batch_size image_size gf_dim df_dim output_c_dim ')
options = OPTIONS._make((batch_size,input_height,ngf,ndf,output_nc))

In [ ]:
#### Start of building the model ####
real_data = tf.placeholder(tf.float32,[None, input_height, input_width,
                                     input_nc + output_nc],name='real_A_and_B_images')

real_A = real_data[:, :, :, :input_nc]
real_B = real_data[:, :, :, input_nc:input_nc + output_nc]

fake_B = generator(real_A, options, False, name="generatorA2B")
fake_A_ = generator(fake_B, options, False, name="generatorB2A")
fake_A = generator(real_B, options, True, name="generatorB2A")
fake_B_ = generator(fake_A, options, True, name="generatorA2B")

DB_fake = discriminator(fake_B, options, reuse=False, name="discriminatorB")
DA_fake = discriminator(fake_A, options, reuse=False, name="discriminatorA")

g_loss_a2b = criterionGAN(DB_fake, tf.ones_like(DB_fake)) \
            + L1_lambda * abs_criterion(real_A, fake_A_) + L1_lambda * abs_criterion(real_B, fake_B_)
g_loss_b2a = criterionGAN(DA_fake, tf.ones_like(DA_fake)) \
            + L1_lambda * abs_criterion(real_A, fake_A_) + L1_lambda * abs_criterion(real_B, fake_B_)
g_loss = criterionGAN(DA_fake, tf.ones_like(DA_fake)) + criterionGAN(DB_fake, tf.ones_like(DB_fake)) \
            + L1_lambda * abs_criterion(real_A, fake_A_) + L1_lambda * abs_criterion(real_B, fake_B_)

fake_A_sample = tf.placeholder(tf.float32,[None, input_height, input_width, input_nc], name='fake_A_sample')
fake_B_sample = tf.placeholder(tf.float32,[None, output_height, output_width, output_nc], name='fake_B_sample')

DB_real = discriminator(real_B, options, reuse=True, name="discriminatorB")
DA_real = discriminator(real_A, options, reuse=True, name="discriminatorA")
DB_fake_sample = discriminator(fake_B_sample, options, reuse=True, name="discriminatorB")
DA_fake_sample = discriminator(fake_A_sample, options, reuse=True, name="discriminatorA")

db_loss_real = criterionGAN(DB_real, tf.ones_like(DB_real))
db_loss_fake = criterionGAN(DB_fake_sample, tf.zeros_like(DB_fake_sample))
db_loss = (db_loss_real + db_loss_fake) / 2

da_loss_real = criterionGAN(DA_real, tf.ones_like(DA_real))
da_loss_fake = criterionGAN(DA_fake_sample, tf.zeros_like(DA_fake_sample))
da_loss = (da_loss_real + da_loss_fake) / 2

d_loss = da_loss + db_loss

g_loss_a2b_sum = tf.summary.scalar("g_loss_a2b", g_loss_a2b)
g_loss_b2a_sum = tf.summary.scalar("g_loss_b2a", g_loss_b2a)
g_loss_sum = tf.summary.scalar("g_loss", g_loss)
g_sum = tf.summary.merge([g_loss_a2b_sum, g_loss_b2a_sum, g_loss_sum])

db_loss_sum = tf.summary.scalar("db_loss", db_loss)
da_loss_sum = tf.summary.scalar("da_loss", da_loss)
d_loss_sum = tf.summary.scalar("d_loss", d_loss)

db_loss_real_sum = tf.summary.scalar("db_loss_real", db_loss_real)
db_loss_fake_sum = tf.summary.scalar("db_loss_fake", db_loss_fake)

da_loss_real_sum = tf.summary.scalar("da_loss_real", da_loss_real)
da_loss_fake_sum = tf.summary.scalar("da_loss_fake", da_loss_fake)

d_sum = tf.summary.merge([da_loss_sum, da_loss_real_sum, da_loss_fake_sum,
                          db_loss_sum, db_loss_real_sum, db_loss_fake_sum, d_loss_sum])

test_A = tf.placeholder(tf.float32,[None, input_height, input_width, input_nc], name='test_A')
test_B = tf.placeholder(tf.float32,[None, output_height, output_width, output_nc], name='test_B')

testB = generator(test_A, options, True, name="generatorA2B")
testA = generator(test_B, options, True, name="generatorB2A")

t_vars = tf.trainable_variables()
d_vars = [var for var in t_vars if 'discriminator' in var.name]
g_vars = [var for var in t_vars if 'generator' in var.name]
for var in t_vars: print(var.name)

generatorA2B/g_e1_c/Conv/weights:0
generatorA2B/g_e1_bn/scale:0
generatorA2B/g_e1_bn/offset:0
generatorA2B/g_e2_c/Conv/weights:0
generatorA2B/g_e2_bn/scale:0
generatorA2B/g_e2_bn/offset:0
generatorA2B/g_e3_c/Conv/weights:0
generatorA2B/g_e3_bn/scale:0
generatorA2B/g_e3_bn/offset:0
generatorA2B/g_r1_c1/Conv/weights:0
generatorA2B/g_r1_bn1/scale:0
generatorA2B/g_r1_bn1/offset:0
generatorA2B/g_r1_c2/Conv/weights:0
generatorA2B/g_r1_bn2/scale:0
generatorA2B/g_r1_bn2/offset:0
generatorA2B/g_r2_c1/Conv/weights:0
generatorA2B/g_r2_bn1/scale:0
generatorA2B/g_r2_bn1/offset:0
generatorA2B/g_r2_c2/Conv/weights:0
generatorA2B/g_r2_bn2/scale:0
generatorA2B/g_r2_bn2/offset:0
generatorA2B/g_r3_c1/Conv/weights:0
generatorA2B/g_r3_bn1/scale:0
generatorA2B/g_r3_bn1/offset:0
generatorA2B/g_r3_c2/Conv/weights:0
generatorA2B/g_r3_bn2/scale:0
generatorA2B/g_r3_bn2/offset:0
generatorA2B/g_r4_c1/Conv/weights:0
generatorA2B/g_r4_bn1/scale:0
generatorA2B/g_r4_bn1/offset:0
generatorA2B/g_r4_c2/Conv/weights:0
gen

In [ ]:
saver = tf.train.Saver()
pool = ImagePool(max_pool)

#### Train cyclegan ####
learn_rate = tf.placeholder(tf.float32, None, name='learning_rate')
d_optim = tf.train.AdamOptimizer(learn_rate, beta1).minimize(d_loss, var_list=d_vars)
g_optim = tf.train.AdamOptimizer(learn_rate, beta1).minimize(g_loss, var_list=g_vars)

init_op = tf.global_variables_initializer()
sess.run(init_op)
writer = tf.summary.FileWriter("./logs", sess.graph)

counter = 1
start_time = time.time()
bar =ProgressBar(redirect_stdout=True)

for epoch in range(epoches):

    dataA = glob('./datasets/{}/*.*'.format(dataset_dir + '/trainA'))
    dataB = glob('./datasets/{}/*.*'.format(dataset_dir + '/trainB'))
    np.random.shuffle(dataA)
    np.random.shuffle(dataB)

    batch_idxs = min(len(dataA), len(dataB)) // batch_size
    lr = init_learn_rate if epoch < epoch_step else init_learn_rate*(epoches-epoch)/(epoches-epoch_step)

    for idx in range(0, batch_idxs):
        batch_files = list(zip(dataA[idx * batch_size:(idx + 1) * batch_size],
                               dataB[idx * batch_size:(idx + 1) * batch_size]))
        batch_images = [load_train_data(batch_file, 286, 256) for batch_file in batch_files]
        batch_images = np.array(batch_images).astype(np.float32)
#             print batch_images

        # Update G network and record fake outputs
        e_fake_A, e_fake_B, _, summary_str = sess.run([fake_A, fake_B, g_optim, g_sum], 
                                                  feed_dict={real_data: batch_images, learn_rate: lr})
        writer.add_summary(summary_str, counter)
        [e_fake_A, e_fake_B] = pool([e_fake_A, e_fake_B])

        # Update D network
        _, summary_str = sess.run([d_optim, d_sum],feed_dict={real_data: batch_images, fake_A_sample: e_fake_A,
                                   fake_B_sample: e_fake_B, learn_rate: lr})
        writer.add_summary(summary_str, counter)

        counter += 1
#         print "Epoch: [%2d]" %epoch ,idx
#         bar.update(idx)
        print(("Epoch: [%2d] [%4d/%4d] time: %4.4f" % (
            epoch, idx, batch_idxs, time.time() - start_time)))

#         if np.mod(epoch, print_freq) == 1:
# #                 sample_model(sample_dir, epoch, idx) ## Print these out instead
#             vdataA = glob('./datasets/{}/*.*'.format(dataset_dir + '/testA'))
#             vdataB = glob('./datasets/{}/*.*'.format(dataset_dir + '/testB'))
#             np.random.shuffle(vdataA)
#             np.random.shuffle(vdataB)
#             vbatch_files = list(zip(vdataA[:batch_size], vdataB[:batch_size]))
#             sample_images = [load_train_data(vbatch_file, is_testing=True) for vbatch_file in vbatch_files]
#             sample_images = np.array(sample_images).astype(np.float32)

#             vfake_A, vfake_B = sess.run([fake_A, fake_B],feed_dict={real_data: sample_images})
#             imgA = imread(vbatch_file[0]) ; imgB = imread(vbatch_file[1])
#             imgA = scipy.misc.imresize(imgA, [256, 256]) ; imgB = scipy.misc.imresize(imgB, [256, 256])

#             ## Start of Plotting
#             plt.figure(figsize=(10,5))
#             plt.subplot(1,2,1); plt.imshow(imgA)
#             plt.title('Original Image')
#             plt.subplot(1,2,2); plt.imshow((vfake_A[0]+1.)/2.)
#             plt.title('Epoch: %d; Style Transferred Image' %epoch )

#             plt.figure(figsize=(10,5))
#             plt.subplot(1,2,1); plt.imshow(imgB)
#             plt.title('Original Image')
#             plt.subplot(1,2,2); plt.imshow((vfake_B[0]+1.)/2.)
#             plt.title('Epoch: %d; Style Transferred Image' %epoch )

#         if np.mod(epoch, save_freq) == 2:
# #                 save(checkpoint_dir, counter)
#             model_name = "cyclegan.model"
#             model_dir = "%s_%s" % (dataset_dir, 256)
#             checkpoint_dir = os.path.join(checkpoint_dir, model_dir)

#             if not os.path.exists(checkpoint_dir):
#                 os.makedirs(checkpoint_dir)
#             saver.save(sess,os.path.join(checkpoint_dir, model_name),global_step=counter)

Epoch: [ 0] [   0/ 832] time: 5.9386
Epoch: [ 0] [   1/ 832] time: 8.3461
Epoch: [ 0] [   2/ 832] time: 9.8465
Epoch: [ 0] [   3/ 832] time: 12.1693
Epoch: [ 0] [   4/ 832] time: 14.8763
Epoch: [ 0] [   5/ 832] time: 17.5811
Epoch: [ 0] [   6/ 832] time: 20.2865
Epoch: [ 0] [   7/ 832] time: 23.2549
Epoch: [ 0] [   8/ 832] time: 26.0238
Epoch: [ 0] [   9/ 832] time: 28.7276
Epoch: [ 0] [  10/ 832] time: 31.1912
Epoch: [ 0] [  11/ 832] time: 33.9285
Epoch: [ 0] [  12/ 832] time: 36.7697
Epoch: [ 0] [  13/ 832] time: 39.4621
Epoch: [ 0] [  14/ 832] time: 42.1861
Epoch: [ 0] [  15/ 832] time: 45.1455
Epoch: [ 0] [  16/ 832] time: 47.8183
Epoch: [ 0] [  17/ 832] time: 50.5111
Epoch: [ 0] [  18/ 832] time: 53.3561
Epoch: [ 0] [  19/ 832] time: 56.1831
Epoch: [ 0] [  20/ 832] time: 58.8984
Epoch: [ 0] [  21/ 832] time: 61.6510
Epoch: [ 0] [  22/ 832] time: 64.3913
Epoch: [ 0] [  23/ 832] time: 67.1163
Epoch: [ 0] [  24/ 832] time: 69.8166
Epoch: [ 0] [  25/ 832] time: 72.4012
Epoch: [ 0] [  

Epoch: [ 0] [ 212/ 832] time: 548.5863
Epoch: [ 0] [ 213/ 832] time: 551.0526
Epoch: [ 0] [ 214/ 832] time: 553.5352
Epoch: [ 0] [ 215/ 832] time: 555.7758
Epoch: [ 0] [ 216/ 832] time: 558.2897
Epoch: [ 0] [ 217/ 832] time: 560.8046
Epoch: [ 0] [ 218/ 832] time: 563.3005
Epoch: [ 0] [ 219/ 832] time: 565.7846
Epoch: [ 0] [ 220/ 832] time: 568.2458
Epoch: [ 0] [ 221/ 832] time: 570.7391
Epoch: [ 0] [ 222/ 832] time: 573.2750
Epoch: [ 0] [ 223/ 832] time: 575.7807
Epoch: [ 0] [ 224/ 832] time: 578.0519
Epoch: [ 0] [ 225/ 832] time: 580.5778
Epoch: [ 0] [ 226/ 832] time: 583.0712
Epoch: [ 0] [ 227/ 832] time: 585.5309
Epoch: [ 0] [ 228/ 832] time: 587.4633
Epoch: [ 0] [ 229/ 832] time: 589.6512
Epoch: [ 0] [ 230/ 832] time: 592.0957
Epoch: [ 0] [ 231/ 832] time: 594.5695
Epoch: [ 0] [ 232/ 832] time: 597.0014
Epoch: [ 0] [ 233/ 832] time: 599.2751
Epoch: [ 0] [ 234/ 832] time: 601.7087
Epoch: [ 0] [ 235/ 832] time: 604.1556
Epoch: [ 0] [ 236/ 832] time: 606.5947
Epoch: [ 0] [ 237/ 832] t

Epoch: [ 0] [ 422/ 832] time: 1083.9379
Epoch: [ 0] [ 423/ 832] time: 1086.5581
Epoch: [ 0] [ 424/ 832] time: 1089.1340
Epoch: [ 0] [ 425/ 832] time: 1091.9872
Epoch: [ 0] [ 426/ 832] time: 1094.6091
Epoch: [ 0] [ 427/ 832] time: 1097.2226
Epoch: [ 0] [ 428/ 832] time: 1099.8514
Epoch: [ 0] [ 429/ 832] time: 1102.4602
Epoch: [ 0] [ 430/ 832] time: 1105.0518
Epoch: [ 0] [ 431/ 832] time: 1107.6440
Epoch: [ 0] [ 432/ 832] time: 1109.9267
Epoch: [ 0] [ 433/ 832] time: 1111.2022
Epoch: [ 0] [ 434/ 832] time: 1113.4722
Epoch: [ 0] [ 435/ 832] time: 1115.9089
Epoch: [ 0] [ 436/ 832] time: 1118.3355
Epoch: [ 0] [ 437/ 832] time: 1120.8079
Epoch: [ 0] [ 438/ 832] time: 1123.2405
Epoch: [ 0] [ 439/ 832] time: 1125.6633
Epoch: [ 0] [ 440/ 832] time: 1128.0974
Epoch: [ 0] [ 441/ 832] time: 1130.3852
Epoch: [ 0] [ 442/ 832] time: 1132.7236
Epoch: [ 0] [ 443/ 832] time: 1135.1796
Epoch: [ 0] [ 444/ 832] time: 1137.6074
Epoch: [ 0] [ 445/ 832] time: 1140.0262
Epoch: [ 0] [ 446/ 832] time: 1142.4521


Epoch: [ 0] [ 627/ 832] time: 1618.9442
Epoch: [ 0] [ 628/ 832] time: 1621.4261
Epoch: [ 0] [ 629/ 832] time: 1623.7710
Epoch: [ 0] [ 630/ 832] time: 1626.1270
Epoch: [ 0] [ 631/ 832] time: 1628.6032
Epoch: [ 0] [ 632/ 832] time: 1631.0831
Epoch: [ 0] [ 633/ 832] time: 1633.5461
Epoch: [ 0] [ 634/ 832] time: 1635.2493
Epoch: [ 0] [ 635/ 832] time: 1637.7973
Epoch: [ 0] [ 636/ 832] time: 1640.6157
Epoch: [ 0] [ 637/ 832] time: 1643.4650
Epoch: [ 0] [ 638/ 832] time: 1646.2501
Epoch: [ 0] [ 639/ 832] time: 1649.1772
Epoch: [ 0] [ 640/ 832] time: 1652.0165
Epoch: [ 0] [ 641/ 832] time: 1654.7676
Epoch: [ 0] [ 642/ 832] time: 1657.5224
Epoch: [ 0] [ 643/ 832] time: 1660.1852
Epoch: [ 0] [ 644/ 832] time: 1663.1216
Epoch: [ 0] [ 645/ 832] time: 1665.9186
Epoch: [ 0] [ 646/ 832] time: 1668.6864
Epoch: [ 0] [ 647/ 832] time: 1671.5274
Epoch: [ 0] [ 648/ 832] time: 1674.4674
Epoch: [ 0] [ 649/ 832] time: 1677.2466
Epoch: [ 0] [ 650/ 832] time: 1680.0679
Epoch: [ 0] [ 651/ 832] time: 1682.9981


Epoch: [ 1] [   0/ 832] time: 2155.2864
Epoch: [ 1] [   1/ 832] time: 2158.3711
Epoch: [ 1] [   2/ 832] time: 2161.0918
Epoch: [ 1] [   3/ 832] time: 2164.0066
Epoch: [ 1] [   4/ 832] time: 2166.8559
Epoch: [ 1] [   5/ 832] time: 2169.6806
Epoch: [ 1] [   6/ 832] time: 2172.5796
Epoch: [ 1] [   7/ 832] time: 2175.6340
Epoch: [ 1] [   8/ 832] time: 2178.4939
Epoch: [ 1] [   9/ 832] time: 2181.3600
Epoch: [ 1] [  10/ 832] time: 2183.9769
Epoch: [ 1] [  11/ 832] time: 2186.8647
Epoch: [ 1] [  12/ 832] time: 2189.7513
Epoch: [ 1] [  13/ 832] time: 2192.6109
Epoch: [ 1] [  14/ 832] time: 2195.4597
Epoch: [ 1] [  15/ 832] time: 2198.3625
Epoch: [ 1] [  16/ 832] time: 2201.2681
Epoch: [ 1] [  17/ 832] time: 2204.0935
Epoch: [ 1] [  18/ 832] time: 2207.1593
Epoch: [ 1] [  19/ 832] time: 2210.1930
Epoch: [ 1] [  20/ 832] time: 2213.0666
Epoch: [ 1] [  21/ 832] time: 2215.9026
Epoch: [ 1] [  22/ 832] time: 2218.7630
Epoch: [ 1] [  23/ 832] time: 2221.4974
Epoch: [ 1] [  24/ 832] time: 2224.3375


...

Epoch: [ 3] [ 796/ 832] time: 6685.1403
Epoch: [ 3] [ 797/ 832] time: 6686.4124
Epoch: [ 3] [ 798/ 832] time: 6687.7006
Epoch: [ 3] [ 799/ 832] time: 6688.9714
Epoch: [ 3] [ 800/ 832] time: 6690.2443
Epoch: [ 3] [ 801/ 832] time: 6691.5182
Epoch: [ 3] [ 802/ 832] time: 6692.8008
Epoch: [ 3] [ 803/ 832] time: 6694.0843
Epoch: [ 3] [ 804/ 832] time: 6695.3648
Epoch: [ 3] [ 805/ 832] time: 6696.6346
Epoch: [ 3] [ 806/ 832] time: 6697.9150
Epoch: [ 3] [ 807/ 832] time: 6699.1841
Epoch: [ 3] [ 808/ 832] time: 6700.4540
Epoch: [ 3] [ 809/ 832] time: 6701.7307
Epoch: [ 3] [ 810/ 832] time: 6702.9991
Epoch: [ 3] [ 811/ 832] time: 6704.2738
Epoch: [ 3] [ 812/ 832] time: 6705.5418
Epoch: [ 3] [ 813/ 832] time: 6706.8150
Epoch: [ 3] [ 814/ 832] time: 6708.0942
Epoch: [ 3] [ 815/ 832] time: 6709.3635
Epoch: [ 3] [ 816/ 832] time: 6710.6378
Epoch: [ 3] [ 817/ 832] time: 6711.9154
Epoch: [ 3] [ 818/ 832] time: 6713.1892
Epoch: [ 3] [ 819/ 832] time: 6714.4595
Epoch: [ 3] [ 820/ 832] time: 6715.7312


Epoch: [ 4] [ 169/ 832] time: 6959.1126
Epoch: [ 4] [ 170/ 832] time: 6960.3792
Epoch: [ 4] [ 171/ 832] time: 6961.6491
Epoch: [ 4] [ 172/ 832] time: 6962.9208
Epoch: [ 4] [ 173/ 832] time: 6964.1890
Epoch: [ 4] [ 174/ 832] time: 6966.0985
Epoch: [ 4] [ 175/ 832] time: 6967.3777
Epoch: [ 4] [ 176/ 832] time: 6968.8809
Epoch: [ 4] [ 177/ 832] time: 6970.1646
Epoch: [ 4] [ 178/ 832] time: 6971.4345
Epoch: [ 4] [ 179/ 832] time: 6973.3472
Epoch: [ 4] [ 180/ 832] time: 6974.6161
Epoch: [ 4] [ 181/ 832] time: 6975.8927
Epoch: [ 4] [ 182/ 832] time: 6977.1629
Epoch: [ 4] [ 183/ 832] time: 6978.6744
Epoch: [ 4] [ 184/ 832] time: 6980.5762
Epoch: [ 4] [ 185/ 832] time: 6981.8573
Epoch: [ 4] [ 186/ 832] time: 6983.1337
Epoch: [ 4] [ 187/ 832] time: 6984.4018
Epoch: [ 4] [ 188/ 832] time: 6985.6737
Epoch: [ 4] [ 189/ 832] time: 6986.9441
Epoch: [ 4] [ 190/ 832] time: 6988.4315
Epoch: [ 4] [ 191/ 832] time: 6989.7055
Epoch: [ 4] [ 192/ 832] time: 6990.9792
Epoch: [ 4] [ 193/ 832] time: 6992.2452


Epoch: [ 4] [ 374/ 832] time: 7226.2689
Epoch: [ 4] [ 375/ 832] time: 7227.5434
Epoch: [ 4] [ 376/ 832] time: 7228.8111
Epoch: [ 4] [ 377/ 832] time: 7230.0880
Epoch: [ 4] [ 378/ 832] time: 7231.3530
Epoch: [ 4] [ 379/ 832] time: 7232.6303
Epoch: [ 4] [ 380/ 832] time: 7233.9078
Epoch: [ 4] [ 381/ 832] time: 7235.1761
Epoch: [ 4] [ 382/ 832] time: 7236.4544
Epoch: [ 4] [ 383/ 832] time: 7237.7268
Epoch: [ 4] [ 384/ 832] time: 7239.0049
Epoch: [ 4] [ 385/ 832] time: 7240.2760
Epoch: [ 4] [ 386/ 832] time: 7241.5394
Epoch: [ 4] [ 387/ 832] time: 7242.8036
Epoch: [ 4] [ 388/ 832] time: 7244.0805
Epoch: [ 4] [ 389/ 832] time: 7245.3487
Epoch: [ 4] [ 390/ 832] time: 7246.6141
Epoch: [ 4] [ 391/ 832] time: 7247.8865
Epoch: [ 4] [ 392/ 832] time: 7249.1633
Epoch: [ 4] [ 393/ 832] time: 7250.4484
Epoch: [ 4] [ 394/ 832] time: 7251.7303
Epoch: [ 4] [ 395/ 832] time: 7253.0040
Epoch: [ 4] [ 396/ 832] time: 7254.2821
Epoch: [ 4] [ 397/ 832] time: 7255.5584
Epoch: [ 4] [ 398/ 832] time: 7256.8257


Epoch: [ 4] [ 579/ 832] time: 7535.1080
Epoch: [ 4] [ 580/ 832] time: 7537.5589
Epoch: [ 4] [ 581/ 832] time: 7539.9978
Epoch: [ 4] [ 582/ 832] time: 7542.2672
Epoch: [ 4] [ 583/ 832] time: 7544.6843
Epoch: [ 4] [ 584/ 832] time: 7547.1416
Epoch: [ 4] [ 585/ 832] time: 7549.5752
Epoch: [ 4] [ 586/ 832] time: 7551.0358
Epoch: [ 4] [ 587/ 832] time: 7552.5029
Epoch: [ 4] [ 588/ 832] time: 7553.7816
Epoch: [ 4] [ 589/ 832] time: 7555.0608
Epoch: [ 4] [ 590/ 832] time: 7556.3706
Epoch: [ 4] [ 591/ 832] time: 7557.8796
Epoch: [ 4] [ 592/ 832] time: 7559.1690
Epoch: [ 4] [ 593/ 832] time: 7560.4375
Epoch: [ 4] [ 594/ 832] time: 7561.8057
Epoch: [ 4] [ 595/ 832] time: 7563.0800
Epoch: [ 4] [ 596/ 832] time: 7564.3605
Epoch: [ 4] [ 597/ 832] time: 7565.9309
Epoch: [ 4] [ 598/ 832] time: 7567.3175
Epoch: [ 4] [ 599/ 832] time: 7568.7456
Epoch: [ 4] [ 600/ 832] time: 7570.0191
Epoch: [ 4] [ 601/ 832] time: 7571.5743
Epoch: [ 4] [ 602/ 832] time: 7572.8533
Epoch: [ 4] [ 603/ 832] time: 7574.2110


Epoch: [ 4] [ 784/ 832] time: 7807.5740
Epoch: [ 4] [ 785/ 832] time: 7808.8539
Epoch: [ 4] [ 786/ 832] time: 7810.1297
Epoch: [ 4] [ 787/ 832] time: 7811.4065
Epoch: [ 4] [ 788/ 832] time: 7812.6762
Epoch: [ 4] [ 789/ 832] time: 7813.9513
Epoch: [ 4] [ 790/ 832] time: 7815.2251
Epoch: [ 4] [ 791/ 832] time: 7816.5009
Epoch: [ 4] [ 792/ 832] time: 7817.7658
Epoch: [ 4] [ 793/ 832] time: 7819.0330
Epoch: [ 4] [ 794/ 832] time: 7820.2999
Epoch: [ 4] [ 795/ 832] time: 7821.5791
Epoch: [ 4] [ 796/ 832] time: 7822.8582
Epoch: [ 4] [ 797/ 832] time: 7824.1260
Epoch: [ 4] [ 798/ 832] time: 7825.4023
Epoch: [ 4] [ 799/ 832] time: 7826.6664
Epoch: [ 4] [ 800/ 832] time: 7827.9356
Epoch: [ 4] [ 801/ 832] time: 7829.2137
Epoch: [ 4] [ 802/ 832] time: 7830.4785
Epoch: [ 4] [ 803/ 832] time: 7831.7419
Epoch: [ 4] [ 804/ 832] time: 7833.0232
Epoch: [ 4] [ 805/ 832] time: 7834.2878
Epoch: [ 4] [ 806/ 832] time: 7835.5608
Epoch: [ 4] [ 807/ 832] time: 7836.8257
Epoch: [ 4] [ 808/ 832] time: 7838.0936


In [ ]:
# Showing all images at the end of the training
vdataA = glob('./datasets/{}/*.*'.format(dataset_dir + '/testA'))
vdataB = glob('./datasets/{}/*.*'.format(dataset_dir + '/testB'))
vbatch_files = list(zip(vdataA, vdataB))
sample_images = [load_train_data(vbatch_file, is_testing=True) for vbatch_file in vbatch_files]
sample_images = np.array(sample_images).astype(np.float32)

vfake_A, vfake_B = sess.run([fake_A, fake_B],feed_dict={real_data: sample_images})
for i, vbatch_file in enumerate(vbatch_files):
    imgA = imread(vbatch_file[0]) ; imgB = imread(vbatch_file[1])
    imgA = scipy.misc.imresize(imgA, [256, 256]) ; imgB = scipy.misc.imresize(imgB, [256, 256])

    ## Start of Plotting
    plt.figure(figsize=(10,5))
    plt.subplot(1,2,1); plt.imshow(imgA)
    plt.title('Original Image No: %d'%i)
    plt.subplot(1,2,2); plt.imshow((vfake_B[i]+1.)/2.)
    plt.title('Style Transferred Image No: %d'%i)

    plt.figure(figsize=(10,5))
    plt.subplot(1,2,1); plt.imshow(imgB)
    plt.title('Original Image No: %d'%i)
    plt.subplot(1,2,2); plt.imshow((vfake_A[i]+1.)/2.)
    plt.title('Style Transferred Image No: %d'%i )
#     break

In [ ]:
sess.close()